In [70]:
import collections
from random import choice
from math import hypot
from socket import socket, AF_INET, SOCK_STREAM
from functools import partial

## Special Methods
data model as a description of Python as a framework.

The special method names allow your objects to implement, support, and interact with
basic language constructs such as:

    • Iteration
    • Collections
    • Attribute access
    • Operator overloading
    • Function and method invocation
    • Object creation and destruction
    • String representation and formatting
    • Managed contexts (i.e., with blocks)

### Named Tuples
the power of implementing just two special methods,__ __getitem____ and __ __len____.

In [11]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
        for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]

In [12]:
c = FrenchDeck()

In [4]:
c.ranks

['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

In [5]:
c.suits

['spades', 'diamonds', 'clubs', 'hearts']

#### __ __len____

In [17]:
c.__len__()

52

In [13]:
len(c)

52

#### __ __getitem____

In [18]:
c.__getitem__(0)

Card(rank='2', suit='spades')

In [15]:
c[0]

Card(rank='2', suit='spades')

#### now it's a sequence data model 

In [30]:
for _ in range(5):
    print(choice(c))

Card(rank='J', suit='clubs')
Card(rank='9', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='J', suit='spades')
Card(rank='10', suit='clubs')


##### Slicing
Because our __ __getitem____ delegates to the [] operator of self._cards, our card automatically
supports slicing.

In [31]:
c[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

##### iterable
Just by implementing the__ __getitem____ special method, our card is also iterable:

In [32]:
for card in c[:3]:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')


In [33]:
for card in reversed(c[:3]):
    print(card)

Card(rank='4', suit='spades')
Card(rank='3', suit='spades')
Card(rank='2', suit='spades')


##### sequential searching of a collection
If a collection has no __ __contains____ method, the in operator does a sequential scan with O(n) time complexity.

In [43]:
print(Card('Q', 'hearts') in c)
print(Card('7', 'beasts') in c)

True
False


##### sorting
attribute access & sorted built-in method

In [44]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [46]:
for card in sorted(c, key=spades_high)[:10]:
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')


#### When to use special methods?
Normally, your code should __not__ have many __direct__ calls to special methods. Unless you
are doing a lot of metaprogramming, you should be implementing special methods
more often than invoking them explicitly.

### Vector

overload ops by __ __add____ and __ __mul____

In [57]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    def __str__(self):
        return '(%r, %r)' % (self.x, self.y)
    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)
    def __abs__(self):
        return hypot(self.x, self.y)
    def __bool__(self):
        return bool(abs(self))
        # return bool(self.x or self.y)
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [58]:
v = Vector(1, 2)

#### __ __repr____
interactive stdout

In [60]:
v

Vector(1, 2)

#### __ __str____
print stdout

If you only implement one of these special methods, choose __ __repr____, because when
no custom __ __str____ is available, Python will call __ __repr____ as a fallback.

In [59]:
print(v)

(1, 2)


#### Arithmetic Operators

In [61]:
a = Vector(1, 1)
b = Vector(2, 3)

In [62]:
a + b

Vector(3, 4)

In [64]:
a * 3

Vector(3, 3)

#### Bool value of a custom type

By default, instances of user-defined classes are considered truthy, unless either
__ __bool____ or __ __len____ is implemented. Basically, bool(x) calls x.__ __bool____() and uses
the result. If __ __bool____ is not implemented, Python tries to invoke x.__ __len____(), and if
that returns zero, bool returns False. Otherwise bool returns True.

In [65]:
bool(v)

True

### Context mangament
the power of __ __enter____() and __ __exit____()

In [67]:
class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [71]:
conn = LazyConnection(('www.python.org', 80))
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes: connection closed

In [74]:
print(resp)

b'HTTP/1.0 301 Moved Permanently\r\nvia: proxy A\r\nDate: Fri, 10 Jul 2020 08:08:46 GMT\r\nServer: Varnish\r\nX-Cache: HIT\r\nX-Timer: S1594368527.837532,VS0,VE0\r\nLocation: https://www.python.org/index.html\r\nConnection: Close\r\nRetry-After: 0\r\nX-Served-By: cache-hkg17933-HKG\r\nX-Cache-Hits: 0\r\nAccept-Ranges: bytes\r\nContent-Length: 0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains\r\n\r\n'
